# Exemplo de uso do pacote JuMP para resolver o TSP

In [1]:
using DelimitedFiles, PyCall

# ========================= #
# Carrega dados do problema #
# ========================= #

# Carregando latitude e longitude dos pontos
LatLon = readdlm("data/sjcLatLon.csv")
LatMin = minimum(LatLon[:,1])
LatMax = maximum(LatLon[:,1])
LonMin = minimum(LatLon[:,2])
LonMax = maximum(LatLon[:,2])

# Carregando a matriz de distâncias
D = readdlm("data/sjcEuclidiano.csv",',')

# Selecionando os n primeiros pontos da matriz de distâncias 
n = 10
D = D[1:n,1:n]

# =============================== #
# Imprimindo o mapa com os pontos #
# =============================== #

# Importa biblioteca python "folium"
flm = pyimport("folium")

# Cria mapa inicial com pontos
mapa = flm.Map()

# Define os limites do mapa de acordo com os pontos
limites = [(LatMin,LonMin),(LatMax,LonMax)]
mapa.fit_bounds(limites)

# Insere os pontos no mapa
idPonto = 1
for i in 1:n
    lat = LatLon[i,1]
    lon = LatLon[i,2]
    flm.Circle(location=[lat,lon],
               color="blue",
               radius=14, 
               tooltip="$idPonto", 
               popup="Lat: $lat \nLon: $lon",
               fill=true,
               fill_opacity=1).add_to(mapa)
    idPonto += 1
end

mapa

PyObject <folium.folium.Map object at 0x00000000022B0EC8>

### Modelo matemático TSP


$\text{Min} \ \ \sum_{ij} d_{ij} x_{ij} \\
\text{Sub.} \ \ x_{ii} = 0 \hspace{10em} , i = 1...n \hspace{2.7em} \text{# Ignora laços} \hspace{14em} \text{(1)} \\
\hspace{1.8em} \sum_{i} x_{ij} = 1 \hspace{8.8em} ,j = 1...n \hspace{2.7em} \text{# Apenas um arco deve chegar em j} \hspace{6.4em} \text{(2)}\\
\hspace{1.8em} \sum_{j} x_{ij} = 1 \hspace{8.8em} ,i = 1...n \hspace{2.7em} \text{# Apenas um arco deve sair de i} \hspace{7.7em} \text{(3)}\\
\hspace{1.8em} n x_{ij} - (n-1) \leq u_{j} - u_{i} \hspace{2.1em} ,i,j = 1...n \hspace{2em} \text{# Elimina subtours (Miller-Tuker-Zemlin)} \hspace{4.2em} \text{(4)}\\
\hspace{1.8em} x_{ij} \in \{0,1\} \\
\hspace{1.8em} u_{i} \in \mathbb{Z} \\
$

In [2]:
using JuMP, CPLEX

model = Model(CPLEX.Optimizer)

@variable(model, x[i in 1:n, j in 1:n], Bin)
@variable(model, u[i in 2:n])

@objective(model, Min, sum(D[i,j] * x[i,j] for i in 1:n, j in 1:n))

# Restrição (1)
for i in 1:n
    @constraint(model, x[i,i] == 0)
end

# Restrição (2)
for j in 1:n
    @constraint(model, sum(x[i,j] for i in 1:n) == 1)
end

# Restrição (3)
for i in 1:n
    @constraint(model, sum(x[i,j] for j in 1:n) == 1)
end

# Restrição (4)
for i in 2:n
    for j in 2:n
        @constraint(model, n * x[i,j] - (n-1) <= u[j] - u[i])
    end
end

# Executa o solver
optimize!(model)

# Pega valor variáveis
x_opt = JuMP.value.(x)
z_opt = JuMP.objective_value(model)

1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 3589.9507.
Tried aggregator 1 time.
MIP Presolve eliminated 19 rows and 10 columns.
Reduced MIP has 92 rows, 99 columns, and 396 nonzeros.
Reduced MIP has 90 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.21 ticks)
Probing time = 0.00 sec. (0.15 ticks)
Tried aggregator 1 time.
Reduced MIP has 92 rows, 99 columns, and 396 nonzeros.
Reduced MIP has 90 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.27 ticks)
Probing time = 0.00 sec. (0.16 ticks)
Clique table members: 56.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.16 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         35

3589.9507291916098

In [3]:
# Exibe a solução no mapa
for i in 1:n
    for j in 1:n
        if x_opt[i,j] > 0.5
            latlonI = (LatLon[i,1],LatLon[i,2])
            latlonJ = (LatLon[j,1],LatLon[j,2])
            flm.PolyLine([ latlonI , latlonJ ],
                popup="$z_opt metros",
                tooltip="$z_opt metros",
                color="blue"
            ).add_to(mapa)
        end
    end
end

mapa

PyObject <folium.folium.Map object at 0x00000000022B0EC8>